In [26]:
import duckdb
import pandas as pd

In [30]:
# working with flight data
flight_sql = "select * from 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/flights.csv'"

In [25]:
flight_df = duckdb.sql(flight_sql)
flight_df.show()

┌───────┬───────────┬────────────┐
│ year  │   month   │ passengers │
│ int64 │  varchar  │   int64    │
├───────┼───────────┼────────────┤
│  1949 │ January   │        112 │
│  1949 │ February  │        118 │
│  1949 │ March     │        132 │
│  1949 │ April     │        129 │
│  1949 │ May       │        121 │
│  1949 │ June      │        135 │
│  1949 │ July      │        148 │
│  1949 │ August    │        148 │
│  1949 │ September │        136 │
│  1949 │ October   │        119 │
│    ·  │   ·       │         ·  │
│    ·  │   ·       │         ·  │
│    ·  │   ·       │         ·  │
│  1960 │ March     │        419 │
│  1960 │ April     │        461 │
│  1960 │ May       │        472 │
│  1960 │ June      │        535 │
│  1960 │ July      │        622 │
│  1960 │ August    │        606 │
│  1960 │ September │        508 │
│  1960 │ October   │        461 │
│  1960 │ November  │        390 │
│  1960 │ December  │        432 │
├───────┴───────────┴────────────┤
│ 144 rows (20 shown

In [31]:
# use of where clause
duckdb.sql("select * from flight_df where month='July'")

┌───────┬─────────┬────────────┐
│ year  │  month  │ passengers │
│ int64 │ varchar │   int64    │
├───────┼─────────┼────────────┤
│  1949 │ July    │        148 │
│  1950 │ July    │        170 │
│  1951 │ July    │        199 │
│  1952 │ July    │        230 │
│  1953 │ July    │        264 │
│  1954 │ July    │        302 │
│  1955 │ July    │        364 │
│  1956 │ July    │        413 │
│  1957 │ July    │        465 │
│  1958 │ July    │        491 │
│  1959 │ July    │        548 │
│  1960 │ July    │        622 │
├───────┴─────────┴────────────┤
│ 12 rows            3 columns │
└──────────────────────────────┘

In [38]:
# use of group by and having where total passengers count is greater than 200
duckdb.sql("select year, sum(passengers) as total_passengers from flight_df group by year having sum(passengers) > 200 order by 1")

┌───────┬──────────────────┐
│ year  │ total_passengers │
│ int64 │      int128      │
├───────┼──────────────────┤
│  1949 │             1520 │
│  1950 │             1676 │
│  1951 │             2042 │
│  1952 │             2364 │
│  1953 │             2700 │
│  1954 │             2867 │
│  1955 │             3408 │
│  1956 │             3939 │
│  1957 │             4421 │
│  1958 │             4572 │
│  1959 │             5140 │
│  1960 │             5714 │
├───────┴──────────────────┤
│ 12 rows        2 columns │
└──────────────────────────┘

In [39]:
# working with Pivot operator
duckdb.sql("SELECT * FROM (PIVOT flight_df ON month USING sum(passengers) GROUP BY year)")

┌───────┬────────┬────────┬──────────┬──────────┬─────────┬────────┬────────┬────────┬────────┬──────────┬─────────┬───────────┐
│ year  │ April  │ August │ December │ February │ January │  July  │  June  │ March  │  May   │ November │ October │ September │
│ int64 │ int128 │ int128 │  int128  │  int128  │ int128  │ int128 │ int128 │ int128 │ int128 │  int128  │ int128  │  int128   │
├───────┼────────┼────────┼──────────┼──────────┼─────────┼────────┼────────┼────────┼────────┼──────────┼─────────┼───────────┤
│  1951 │    163 │    199 │      166 │      150 │     145 │    199 │    178 │    178 │    172 │      146 │     162 │       184 │
│  1960 │    461 │    606 │      432 │      391 │     417 │    622 │    535 │    419 │    472 │      390 │     461 │       508 │
│  1954 │    227 │    293 │      229 │      188 │     204 │    302 │    264 │    235 │    234 │      203 │     229 │       259 │
│  1958 │    348 │    505 │      337 │      318 │     340 │    491 │    435 │    362 │    363 │  

In [44]:
# generating row numbers
duckdb.sql("select * , row_number() over( partition by year order by year desc) as rn from flight_df")

┌───────┬───────────┬────────────┬───────┐
│ year  │   month   │ passengers │  rn   │
│ int64 │  varchar  │   int64    │ int64 │
├───────┼───────────┼────────────┼───────┤
│  1955 │ January   │        242 │     1 │
│  1955 │ February  │        233 │     2 │
│  1955 │ March     │        267 │     3 │
│  1955 │ April     │        269 │     4 │
│  1955 │ May       │        270 │     5 │
│  1955 │ June      │        315 │     6 │
│  1955 │ July      │        364 │     7 │
│  1955 │ August    │        347 │     8 │
│  1955 │ September │        312 │     9 │
│  1955 │ October   │        274 │    10 │
│    ·  │   ·       │         ·  │     · │
│    ·  │   ·       │         ·  │     · │
│    ·  │   ·       │         ·  │     · │
│  1958 │ March     │        362 │     3 │
│  1958 │ April     │        348 │     4 │
│  1958 │ May       │        363 │     5 │
│  1958 │ June      │        435 │     6 │
│  1958 │ July      │        491 │     7 │
│  1958 │ August    │        505 │     8 │
│  1958 │ S